<a href="https://colab.research.google.com/github/linhvien/Stock-analysis/blob/main/Vietnamese_stock_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import libraries

In [1]:
from bs4 import BeautifulSoup
import requests

import re
import json
import csv
from io import StringIO
from datetime import datetime, timedelta
import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Overview

In [25]:
start_date = '2019-01-01'
end_date = str(datetime.now().strftime('%Y-%m-%d'))
stock = 'VTV'
portfolio = ''
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}

#Financial data

Loading financial data of Vietnamese stocks from api vndirect

In [23]:
class Financialdata:
  url_BS = 'https://finfo-api.vndirect.com.vn/v3/stocks/financialStatement?secCodes={}&reportTypes=QUARTER&modelTypes=1,89,101,411&fromDate={}&toDate={}'
  url_IS = 'https://finfo-api.vndirect.com.vn/v3/stocks/financialStatement?secCodes={}&reportTypes=QUARTER&modelTypes=2,90,102,412&fromDate={}&toDate={}'
  url_CF = 'https://finfo-api.vndirect.com.vn/v3/stocks/financialStatement?secCodes={}&reportTypes=QUARTER&modelTypes=3,91,103,413&fromDate={}&toDate={}'
  url_ratio = 'https://finfo-api.vndirect.com.vn/v4/ratios?q=code:{}~itemCode:53030,52005,51050,53021,52001,52002,54018,712010,712020,712030,712040~reportDate:{}'
  
  def __init__(self,stock,start_date,end_date,*arg, **karg):
    self.stock = stock
    self.start_date = start_date
    self.end_date = end_date
    self.session = requests.Session()

# Get balance sheet
  def get_BS(self):
    page = self.session.get(self.url_BS.format(self.stock,self.start_date,self.end_date), headers=headers)
    data = page.json()

    data_dates = {}
    for item in data['data']['hits']:
      item = item['_source']
      date = item['fiscalDate']
      itemName = item['itemName']
      numericValue = item['numericValue']
      if date not in data_dates:
        data_dates[date] = [[], []]
      else:
        if itemName not in data_dates[date][0]:
          data_dates[date][0].append(itemName)
          data_dates[date][1].append(numericValue)

    for i, (date, item) in enumerate(data_dates.items()):
      df_date = pd.DataFrame(data={'index':item[0], date[:7]:item[1]})
      if i == 0:
        df = df_date
      else:
        df = pd.merge(df, df_date, how='inner')
    return df

# get income statement
  def get_IS(self):
    page = self.session.get(self.url_IS.format(self.stock,self.start_date,self.end_date), headers=headers)
    data = page.json()

    data_dates = {}
    for item in data['data']['hits']:
      item = item['_source']
      date = item['fiscalDate']
      itemName = item['itemName']
      numericValue = item['numericValue']
      if date not in data_dates:
        data_dates[date] = [[], []]
      else:
        if itemName not in data_dates[date][0]:
          data_dates[date][0].append(itemName)
          data_dates[date][1].append(numericValue)

    for i, (date, item) in enumerate(data_dates.items()):
      df_date = pd.DataFrame(data={'index':item[0], date[:7]:item[1]})
      if i == 0:
        df = df_date
      else:
        df = pd.merge(df, df_date, how='inner')
    return df

# get cashflow statement
  def get_CF(self):
    page = self.session.get(self.url_CF.format(self.stock,self.start_date,self.end_date), headers=headers)
    data = page.json()

    data_dates = {}
    for item in data['data']['hits']:
      item = item['_source']
      date = item['fiscalDate']
      itemName = item['itemName']
      numericValue = item['numericValue']
      if date not in data_dates:
        data_dates[date] = [[], []]
      else:
        if itemName not in data_dates[date][0]:
          data_dates[date][0].append(itemName)
          data_dates[date][1].append(numericValue)

    for i, (date, item) in enumerate(data_dates.items()):
      df_date = pd.DataFrame(data={'index':item[0], date[:7]:item[1]})
      if i == 0:
        df = df_date
      else:
        df = pd.merge(df, df_date, how='inner')
    return df

In [20]:
BS_data = Financialdata(stock,start_date,end_date).get_BS()
BS_data.to_csv(r'BS_{}.csv'.format(stock),index=False)
BS_data.head()

,index,2021-06,2021-03,2020-12,2020-09,2020-06,2020-03,2019-12,2019-09,2019-06,2019-03
0,Tiền và các khoản tương đương tiền,1663036014269,1197956795582,2111242815581,2335881481579,2176667357252,1379711866875,2665194638452,1043473767363,1676491290318,1209154406312
1,Tiền,1261476873446,869981048166,863853260384,995896481579,2044667357252,1028411866875,2378583764655,987873767363,826491290318,1029154406312
2,Các khoản tương đương tiền,401559140823,327975747416,1247389555197,1339985000000,132000000000,351300000000,286610873797,55600000000,850000000000,180000000000
3,Các khoản đầu tư tài chính ngắn hạn,19971222710510,19542794237097,17313679774893,17872002664622,16363060738340,14370288608530,12435744328964,10238285848104,9306531479431,9666846652579
4,Đầu tư ngắn hạn,1129553032,1123201739,1124178861,1128087349,1129064471,1148118350,1153041048,443154506731,443157926658,443154995292


In [21]:
IS_data = Financialdata(stock,start_date,end_date).get_IS()
IS_data.to_csv(r'IS_{}.csv'.format(stock),index=False)
IS_data.head()

,index,2021-06,2021-03,2020-12,2020-09,2020-06,2020-03,2019-12,2019-09,2019-06,2019-03
0,Các khoản giảm trừ doanh thu,12980283496,50444960886,5105999997,14071686928,14352408276,53092072488,16738916476,13142547924,11441072483,40784427090
1,Doanh thu thuần,15715806166465,13190270122852,14424794514944,15563157998229,15495233758799,14153099953575,14238961478162,14290900447018,14598960549757,13189300287807
2,Giá vốn hàng bán,8861448981220,7435389686438,7766620813396,8296414517093,8357339161700,7547288345650,7558658856714,7567934476550,7585015000599,7034297778254
3,Lợi nhuận gộp,6854357185245,5754880436414,6658173701548,7266743481136,7137894597099,6605811607925,6680302621448,6722965970468,7013945549158,6155002509553
4,Doanh thu hoạt động tài chính,292767071141,284839944505,602270330836,358591147428,335351494778,284879682275,233576605377,206732692690,190720049829,176287359587


In [26]:
CF_data = Financialdata(stock,start_date,end_date).get_CF()
CF_data.to_csv(r'CF_{}.csv'.format(stock),index=False)
CF_data.head()

,index,2021-06,2021-03,2020-12,2020-09,2020-06,2020-03,2019-12,2019-09,2019-06,2019-03
0,Chi phí khấu hao tài sản cố định,3031474297,3033645507,3033277726,3046833295,3060036210,3063586821,2868758280,2917196588,2984545577,2541628920
1,Phân bổ lợi thế thương mại,0,0,0,0,0,0,0,0,0,0
2,"Dự phòng giảm giá các khoản đầu tư ngắn hạn, d...",0,0,0,0,0,0,0,0,0,0
3,"Lãi, lỗ chênh lệch tỷ giá hối đoái chưa thực hiện",0,0,0,0,0,0,-1303130,2710752,-1667431,259809
4,Lãi/(lỗ) từ thanh lý tài sản cố định,0,0,0,0,0,0,0,0,0,0


#Stock analysis

In [34]:
r = requests.get('https://finance.vietstock.vn/nganh/1-ban-buon.htm',headers=headers)
df = pd.read_html(r.text)
pd.DataFrame(df[1])

,STT,Mã CK,Sàn,Ngành cấp 3,KLCPLH,Giá 1 ngày,Giá 5 ngày
0,1,AAT,HOSE,"Bán buôn quần áo, vải và vật tư liên quan",34800000,"22,600 +600(+2.73%)",22550
1,2,AAV,HNX,"Bán buôn thực phẩm, tạp hóa và các sản phẩm có...",36656180,"26,400 +300(+1.15%)",25700
2,3,ABS,HOSE,Bán buôn dầu và các sản phẩm dầu khí,80000000,"20,100 +1,300(+6.91%)",19800
3,4,AMV,HNX,"Bán buôn các thiết bị, vật tư chuyên môn và th...",91108861,"11,200 +400(+3.70%)",11400
4,5,ARM,HNX,"Bán buôn máy móc, thiết bị và vật tư",3111283,"48,000 +3,400(+7.62%)",50000
5,6,CKV,HNX,"Bán buôn máy móc, thiết bị và vật tư",4012000,"14,000 0(0%)",14100
6,7,CLM,HNX,Bán buôn kim loại và khoáng sản (trừ dầu khí),11000000,"21,000 0(0%)",21000
7,8,CMC,HNX,"Bán buôn máy móc, thiết bị và vật tư",4561050,"7,200 +600(+9.09%)",5600
8,9,CVN,HNX,Bán buôn kim loại và khoáng sản (trừ dầu khí),19799994,"9,600 +800(+9.09%)",9100
9,10,DBT,HOSE,Bán buôn dược phẩm và tạp hóa dược,14205116,"15,900 -300(-1.85%)",17200
